## Выделение аномалией по эффективности в процессах.

In [213]:
import pandas as pd
import numpy as np

In [214]:
df = pd.read_csv('Event_log.txt', sep='\t', encoding='cp1251')

In [215]:
df['Event end'] = pd.to_datetime(df['Event end'])
df = df.sort_values(by='Event end')

In [216]:
df.head()

,CaseID,Event end,Activity,Activity Category,Activity Class,Activity type,User,Department,Amount,Purchase type,Supplier,Supplier risk level,Supplier Size,Supplier_type,INVOICE,Cleared invoice,PO TYPE,Material,User_type
57795,1 - 478,2017-01-03 10:21:02,Заказ на поставку создан,Заказ на поставку создан,Заказ на поставку,Заказ на поставку создан,User_43,Uncategorized,1.163763e+07,Товары,Leisure Clearing House,Low,Large,Внутренние,NaN,NaN,Заказ на оформление закупки,Green Lock Nut 3,A
50372,1 - 478,2017-01-03 10:21:20,Заказ на поставку: изменен статус выпуска: Дан...,Заказ на поставку изменен,Заказ на поставку,Заказ на поставку: изменен статус выпуска: воз...,User_43,Uncategorized,1.163763e+07,Товары,Leisure Clearing House,Low,Large,Внутренние,NaN,NaN,Заказ на оформление закупки,Green Lock Nut 3,A
50373,1 - 478,2017-01-03 10:21:20,Заказ на поставку согласован 2,Заказ на поставку согласован,Заказ на поставку,Заказ на поставку согласован 2,User_43,Uncategorized,1.163763e+07,Товары,Leisure Clearing House,Low,Large,Внутренние,NaN,NaN,Заказ на оформление закупки,Green Lock Nut 3,A
57796,1 - 478,2017-01-03 10:24:36,Заказ на поставку изменен: увеличена стоимость,Заказ на поставку изменен,Заказ на поставку,Заказ на поставку изменен,User_43,Uncategorized,1.163763e+07,Товары,Leisure Clearing House,Low,Large,Внутренние,NaN,NaN,Заказ на оформление закупки,Green Lock Nut 3,A
57797,1 - 478,2017-01-03 10:24:36,Заказ на поставку изменен: эффективная стоимость,Заказ на поставку изменен,Заказ на поставку,Заказ на поставку изменен,User_43,Uncategorized,1.163763e+07,Товары,Leisure Clearing House,Low,Large,Внутренние,NaN,NaN,Заказ на оформление закупки,Green Lock Nut 3,A


Рассмотрим на уровне класса события ("Activity Class")

In [217]:
df['Activity Class'].unique()

array(['Заказ на поставку', 'Поступление материала', 'Заявка', 'Счет',
       'Авансовый платеж', 'Платеж (выравнивание)', 'Учет услуг'],
      dtype=object)

Создадим матрицу признаков *features*, в которой будем хранить описание для каждой цепочки.

Матрица состоит из массивов(строк), которые отвечают своей цепочке(объекту). Столбцы матрицы отвечают классам событий.

На *i*-ой позиции такого массива(строки матрицы *features*) располагается количество событий, произошедших в рамках *i*-ого класса событий из массива классов событий *classes*. Порядок классов в массиве *classes* такой же, как и в выводе ячейки выше.

In [218]:
cases = df['CaseID'].unique()
classes = df['Activity Class'].unique()
features = np.zeros((len(cases), len(classes)))

for i in range(len(cases)):
    chain = df[df['CaseID'] == cases[i]]    
    chain_count = chain.groupby('Activity Class')['Activity'].value_counts()
    for j in range(len(classes)):
        if classes[j] in chain_count:
            features[i][j] = chain_count[classes[j]].sum()

In [219]:
features.shape

(8025, 7)

Транспонируем матрицу *features* и подсчитаем среднее значение повторений для каждого класса.

In [220]:
mean_values = np.zeros(len(classes))


tran_feat = features.T

for i in range(len(classes)):
    mean_values[i] = tran_feat[i].mean()

In [221]:
fin_dict = dict(zip(classes, mean_values))
fin_dict

{'Авансовый платеж': 0.1611214953271028,
 'Заказ на поставку': 2.8547040498442366,
 'Заявка': 0.7862928348909657,
 'Платеж (выравнивание)': 1.573208722741433,
 'Поступление материала': 1.1025545171339564,
 'Счет': 4.099314641744548,
 'Учет услуг': 0.4181931464174455}

Выделим аномальные цепочки.

Аномальной цепочкой назовем такую цепочку, которая хотя бы по одному классу превышает среднее количество повторений.

Создадим матрицу *anomaly*, в которой будем хранить 1 на (*i*, *j*) позиции, если у *i*-ого объекта *j*-ый класс превышает среднее количество повторений, и 0 иначе.

In [222]:
anomaly = np.zeros((len(cases), len(classes)))

for j in range(len(classes)):
    anomaly[np.where(tran_feat[j] > mean_values[j])[0]] = 1

Теперь посмотрим, какую долю от общего числа цепочек занимают аномалии.

In [223]:
len(np.unique(np.where(anomaly == 1)[0])) / len(cases)

0.5753271028037383

Рассмотрим другой подход, где от среднего отсупается одно стандартное отклонение.

In [224]:
anomaly_1 = np.zeros((len(cases), len(classes)))

for j in range(len(classes)):
    anomaly_1[np.where(tran_feat[j] > mean_values[j] + tran_feat[j].std())[0]] = 1

In [225]:
len(np.unique(np.where(anomaly_1 == 1)[0])) / len(cases)

0.3069158878504673

Доля уменьшилась соответственно.

Возьмем теперь два стандартных отклонения.

In [226]:
anomaly_2 = np.zeros((len(cases), len(classes)))

for j in range(len(classes)):
    anomaly_2[np.where(tran_feat[j] > mean_values[j] +2 * tran_feat[j].std())[0]] = 1

In [227]:
len(np.unique(np.where(anomaly_2 == 1)[0])) / len(cases)

0.20809968847352026